In [11]:
import torch
import random

In [135]:
def net2net_step(Wt1, Wt2,  inp, teacher_hidden, student_hidden, out_num, debug=False):
    print ('inp', inp)
    print ('t hid', teacher_hidden)
    print ('s hid', student_hidden)
    print ('out_num', out_num)
    mapping_num = [1]*teacher_hidden
    mapping = {}

    Ws1 = torch.randn(inp,student_hidden)
    Ws2 = torch.randn(student_hidden,out_num)
    
    Ws1[:,:teacher_hidden] = Wt1[:,:teacher_hidden]
    for i in range(teacher_hidden, student_hidden):
        if debug:
            id = teacher_hidden-1
        else:
            id = random.randint(0, teacher_hidden-1)
        mapping_num[id]+=1
        mapping[i] = id
        Ws1[:,i] = Wt1[:,id]

    Ws2[:teacher_hidden] = Wt2[:teacher_hidden]
    for i in range(student_hidden):
        id = mapping.get(i, i)
        num = mapping_num[id] 
        Ws2[i] = Wt2[id]/(num)
    return Ws1, Ws2

def net2net_mlp(mlp1, mlp2):
    old_Ws2 = None
    new_matrices = []
    for i in range(len(mlp1)-1):
        if old_Ws2 is None:
            Wt1 = mlp1[i].weight.T
            Wt2 = mlp1[i+1].weight.T
        else:
            Wt1 = old_Ws2
            Wt2 = mlp1[i+1].weight.T
            
        
        Ws1, Ws2 = net2net_step(Wt1, Wt2,  Wt1.shape[0],
                                Wt1.shape[1],
                                mlp2[i].weight.T.shape[1], 
                                Wt2.shape[1])
        mlp2[i].weight.data = Ws1.T
        mlp2[i+1].weight.data = Ws2.T
        old_Ws2 = Ws2
        
        

In [134]:
def net2net_step(Wt1, Wt2,  inp, teacher_hidden, student_hidden, out_num, debug=False):
    print ('inp', inp)
    print ('t hid', teacher_hidden)
    print ('s hid', student_hidden)
    print ('out_num', out_num)
    mapping_num = [1]*teacher_hidden
    mapping = {}

    Ws1 = torch.randn(inp,student_hidden)
    Ws2 = torch.randn(student_hidden,out_num)
    
    Ws1[:,:teacher_hidden] = Wt1[:,:teacher_hidden]
    for i in range(teacher_hidden, student_hidden):
        if debug:
            id = teacher_hidden-1
        else:
            id = random.randint(0, teacher_hidden-1)
        mapping_num[id]+=1
        mapping[i] = id
        Ws1[:,i] = Wt1[:,id]

    Ws2[:teacher_hidden] = Wt2[:teacher_hidden]
    for i in range(student_hidden):
        id = mapping.get(i, i)
        num = mapping_num[id] 
        Ws2[i] = Wt2[id]/(num)
    return Ws1, Ws2

def net2net_mlp(mlp1, mlp2):
    old_Ws2 = None
    new_matrices = []
    for i in range(len(mlp1)-1):
        if old_Ws2 is None:
            Wt1 = torch.cat([mlp1[i].weight.T, mlp1[i].bias.view(-1, 1).T])
            Wt2 = torch.cat([mlp1[i+1].weight.T, mlp1[i+1].bias.view(1, -1)])
            planned_Ws1 = torch.cat([mlp2[i].weight.T, mlp2[i].bias.view(1, -1)])
        else:
            Wt1 = old_Ws2
            Wt2 = mlp1[i+1].weight.T
            planned_Ws1 = mlp2[i].weight.T
            
        print (Wt1.shape, Wt2.shape)
        Ws1, Ws2 = net2net_step(Wt1, Wt2,  Wt1.shape[0],
                                Wt1.shape[1] + 1,# bias
                                planned_Ws1.shape[1] + 1, #bias 
                                Wt2.shape[1])
        print (Ws1.shape, Ws2.shape)
        #mlp2[i].weight.data = Ws1[:-1].T
        #mlp2[i+1].weight.data = Ws2[:-1].T
        #old_Ws2 = Ws2
        
M1 = torch.nn.Sequential(torch.nn.Linear(128, 64, bias=True),
                        torch.nn.Linear(64, 10, bias=True))
M2 = torch.nn.Sequential(torch.nn.Linear(128, 96, bias=True),
                        torch.nn.Linear(96, 10, bias=True))
#net2net_mlp(M1, M2)

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 128 but got size 64 for tensor number 1 in the list.

In [81]:
M1 = torch.nn.Sequential(torch.nn.Linear(128, 64, bias=False),
                        torch.nn.Linear(64, 10, bias=False))
M2 = torch.nn.Sequential(torch.nn.Linear(128, 96, bias=False),
                        torch.nn.Linear(96, 10, bias=False))
net2net_mlp(M1, M2)

TypeError: expected Tensor as element 1 in argument 0, but got NoneType

In [128]:
M1 = torch.nn.Sequential(torch.nn.Linear(128, 64, bias=False),
                         torch.nn.Linear(64, 32, bias=False),
                         torch.nn.Linear(32, 16, bias=False),
                        torch.nn.Linear(16, 10, bias=False))
M2 = torch.nn.Sequential(torch.nn.Linear(128, 96, bias=False),
                         torch.nn.Linear(96, 48, bias=False),
                         torch.nn.Linear(48, 24, bias=False),
                        torch.nn.Linear(24, 10, bias=False))
#net2net_mlp(M1, M2)

In [98]:
x = torch.rand(256, 128)
abs(M1(x) - M2(x)).sum()/256

RuntimeError: mat1 and mat2 shapes cannot be multiplied (256x96 and 95x10)

In [35]:
for m in M2:
    print (m.weight.shape)

torch.Size([96, 128])
torch.Size([48, 96])
torch.Size([24, 48])
torch.Size([10, 24])


In [44]:
for m in M2:
    print (m.weight.shape)

torch.Size([96, 128])
torch.Size([32, 96])
torch.Size([24, 48])
torch.Size([10, 24])


In [133]:
M1[0]

Sequential(
  (0): Linear(in_features=128, out_features=64, bias=False)
  (1): Linear(in_features=64, out_features=32, bias=False)
  (2): Linear(in_features=32, out_features=16, bias=False)
  (3): Linear(in_features=16, out_features=10, bias=False)
)